In [208]:
# region General Imports
import os
import uuid
import shutil
import time
import random
import datetime
import glob
import pickle
import tqdm
import copy
import optuna
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import mne
from rich import print as rprint
from rich.pretty import pprint as rpprint
from tqdm import tqdm
from itertools import chain
from functools import partial
# endregion General Imports

from model_optim.model_optimizer import ModelOptimizer

# Dataset
from custom_datasets.fatigue_mi import FatigueMI

%load_ext autoreload
%autoreload 3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [213]:
model_optimizer = ModelOptimizer(
    dataset=FatigueMI(),
    model_name="shallow_conv_net"
)

In [214]:
study = model_optimizer.search_best_model(
    subjects=[2],
    # channels=[],
    max_iter = 50,
    max_epochs=5,
    max_stag_count=25,
    rounds=1,
    replace_previous_study_for_subjects=True
)

  0%|          | 0/50 [00:00<?, ?it/s]

Sampling frequency of the instance is already 300.0, returning unmodified.
Adding metadata with 3 columns
Epoch 1/5


/home/arazzz/anaconda3/envs/moabb_model_optimization/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.4354 - loss: 99.5273 - val_accuracy: 0.5000 - val_loss: 73.6332 - learning_rate: 0.0010
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5602 - loss: 69.8405 - val_accuracy: 0.3889 - val_loss: 57.4031 - learning_rate: 0.0010
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6326 - loss: 54.9217 - val_accuracy: 0.6667 - val_loss: 46.3352 - learning_rate: 0.0010
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5901 - loss: 44.4146 - val_accuracy: 0.3889 - val_loss: 37.9287 - learning_rate: 0.0010
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6659 - loss: 36.3195 - val_accuracy: 0.4444 - val_loss: 31.0214 - learning_rate: 0.0010


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 666ms/step - accuracy: 0.4545 - loss: 31.0747


Adding metadata with 3 columns
Epoch 1/5


/home/arazzz/anaconda3/envs/moabb_model_optimization/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.5000 - loss: 144.1823 - val_accuracy: 0.5556 - val_loss: 115.2957 - learning_rate: 0.0010
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.7500 - loss: 115.1503 - val_accuracy: 0.5556 - val_loss: 98.3325 - learning_rate: 0.0010
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.8382 - loss: 98.0597 - val_accuracy: 0.6111 - val_loss: 85.8846 - learning_rate: 0.0010
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.8971 - loss: 85.6764 - val_accuracy: 0.6111 - val_loss: 76.1273 - learning_rate: 0.0010
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.9706 - loss: 75.8021 - val_accuracy: 0.6667 - val_loss: 67.9432 - learning_rate: 0.0010


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 673ms/step - accuracy: 0.4091 - loss: 68.0563


Adding metadata with 3 columns


/home/arazzz/anaconda3/envs/moabb_model_optimization/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.3382 - loss: 147.4602 - val_accuracy: 0.3889 - val_loss: 120.0751 - learning_rate: 0.0010
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.7500 - loss: 119.8688 - val_accuracy: 0.5556 - val_loss: 103.3427 - learning_rate: 0.0010
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.8824 - loss: 103.0932 - val_accuracy: 0.4444 - val_loss: 91.2861 - learning_rate: 0.0010
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.9412 - loss: 90.7711 - val_accuracy: 0.5000 - val_loss: 81.7075 - learning_rate: 0.0010
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.9559 - loss: 80.9942 - val_accuracy: 0.5000 - val_loss: 74.1207 - learning_rate: 0.0010


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 698ms/step - accuracy: 0.5000 - loss: 74.0352


Adding metadata with 3 columns
Epoch 1/5


/home/arazzz/anaconda3/envs/moabb_model_optimization/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.5147 - loss: 216.8327 - val_accuracy: 0.4444 - val_loss: 169.9391 - learning_rate: 0.0010
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.6912 - loss: 169.8374 - val_accuracy: 0.5000 - val_loss: 142.2037 - learning_rate: 0.0010
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.6618 - loss: 141.9864 - val_accuracy: 0.4444 - val_loss: 122.3098 - learning_rate: 0.0010
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.6912 - loss: 122.1379 - val_accuracy: 0.3889 - val_loss: 106.8416 - learning_rate: 0.0010
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.7353 - loss: 106.6416 - val_accuracy: 0.5556 - val_loss: 94.4642 - learning_rate: 0.0010


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 692ms/step - accuracy: 0.5000 - loss: 94.4608




In [215]:
model_optimizer.get_study_metrics(study).sort_values(by="scores", ascending=True)
# model_optimizer.get_study_metrics(study).sort_values(by="test_acc", ascending=False)

,train_acc,test_acc,val_acc,train_val_diff,scores,channels_selected
41,0.808824,0.590909,0.833333,0.024510,0.167317,"[P3, C3, Fz, F4, P4, Pz, Fp1, Fp2, T5, O1, O2,..."
45,0.911765,0.590909,0.777778,0.133987,0.222872,"[P3, C3, Fz, F4, P4, Pz, Fp1, Fp2, T5, O1, O2,..."
25,0.764706,0.272727,0.777778,0.013072,0.222872,"[P3, C3, F3, C4, Cz, Pz, Fp2, T3, T5, O2, F7, ..."
39,0.852941,0.636364,0.722222,0.130719,0.278428,"[P3, C3, Fz, F4, P4, Pz, Fp1, Fp2, T5, O1, O2,..."
42,0.838235,0.590909,0.722222,0.116013,0.278428,"[P3, C3, Fz, F4, P4, Pz, Fp1, Fp2, T5, O1, O2,..."
2,0.941176,0.500000,0.666667,0.274510,0.333783,"[P3, C3, F3, C4, Pz, Fp2, T5, O2, T6]"
47,0.970588,0.409091,0.666667,0.303922,0.333933,"[C3, Fz, F4, P4, Pz, Fp1, Fp2, T5, O1, O2, F7,..."
29,0.970588,0.545455,0.666667,0.303922,0.333983,"[P3, C3, Fz, C4, P4, Cz, Pz, Fp1, Fp2, T5, O1,..."
11,0.573529,0.363636,0.666667,0.093137,0.333983,"[P3, C3, F3, C4, Cz, Pz, Fp2, T3, T5, O2, F7, ..."
0,0.691176,0.454545,0.666667,0.024510,0.333983,"[P3, C3, Fz, Cz, Pz, Fp2, T3, T5, O1, O2, F7, ..."


In [216]:
rpprint({ k: v for k, v in study.best_trial.params.items() if not k.startswith("channels") })
rprint("test_accuracy =", study.best_trial.user_attrs["trial_data"]["test_accuracy"])
rprint("val_accuracy =", np.max(study.best_trial.user_attrs["trial_data"]["val_accuracy"]))
rprint("channels_selected =", study.best_trial.user_attrs["trial_data"]["channels_selected"])

{
│   'sfreq': 256,
│   'batch_size': 160,
│   'pool_size_d2': 65,
│   'strides_d2': 29,
│   'conv_filters_d2': 31,
│   'conv2d_1_units': 120,
│   'conv2d_2_units': 110,
│   'l2_reg_1': 0.2864980963607602,
│   'l2_reg_2': 0.6518631293670971,
│   'l2_reg_3': 0.05624824674993861,
│   'dropout_rate': 0.8
}

test_accuracy = 0.5909091234207153

val_accuracy = 0.8333333134651184

channels_selected = ['P3' 'C3' 'Fz' 'F4' 'P4' 'Pz' 'Fp1' 'Fp2' 'T5' 'O1' 'O2' 'F7' 'T6']